# SQL


Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание проекта

**Заказчик исследования:** Продакт-менеджер (ФИО, департамент), ответственный за продвижение продукта и увеличение вовлеченности аудитории.

**Цель исследования:** Сформулировать ценностное предложение для нового продукта.

**Задача аналитику:** Проанализировать базу данных

**Данные для исследования:** База данных: информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

## Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Содержание проекта
### 1. Подключение к базе данных
### 2. Изучение таблиц
### 3. Запросы по заданиям
### 4. Выводы 

### 1. Подключение к базе данных


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### 2. Изучение таблиц


In [3]:
# Выведем по 5 строк каждой из таблиц, таблица books:
books_head = ''' SELECT *
            FROM books
            LIMIT 5
        '''
pd.io.sql.read_sql(books_head, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# таблица authors
authors_head = ''' SELECT *
            FROM authors
            LIMIT 5
        '''
pd.io.sql.read_sql(authors_head, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# таблица ratings
ratings_head = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''
pd.io.sql.read_sql(ratings_head, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# таблица reviews
reviews_head = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''
pd.io.sql.read_sql(reviews_head, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
# таблица publishers
publishers_head = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''
pd.io.sql.read_sql(publishers_head, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### 3. Запросы по заданиям


**Cколько книг вышло после 1 января 2000 года**

In [8]:
select_1 = ''' SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '01.01.2000'
        '''
pd.io.sql.read_sql(select_1, con = engine)

,count
0,819


**Количество обзоров и среднюю оценку для каждой книги**

In [9]:
select_2 = """
SELECT 
    b.title, 
    COUNT(DISTINCT rv.review_id) AS reviews_count, 
    AVG(r.rating) AS average_rating
FROM 
    books b
    LEFT JOIN ratings r ON b.book_id = r.book_id
    LEFT JOIN reviews rv ON b.book_id = rv.book_id
GROUP BY 
    b.book_id
ORDER BY reviews_count DESC;
"""
pd.io.sql.read_sql(select_2, con = engine)

,title,reviews_count,average_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


**Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключение из анализа брошюр)**

In [10]:
select_3 = ''' 
SELECT 
    top.publisher_id,
    top.publisher,
    top.number_books
FROM 
    (SELECT
         b.publisher_id AS publisher_id,
         COUNT(b.book_id) AS number_books,
         p.publisher AS publisher
     FROM 
         books AS b
         LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
     WHERE 
         num_pages > 50
     GROUP BY 
         b.publisher_id,
         p.publisher
     ORDER BY    
             number_books DESC) AS top
LIMIT 1
; '''

pd.io.sql.read_sql(select_3, con = engine)

,publisher_id,publisher,number_books
0,212,Penguin Books,42


**Определить автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками**


In [14]:
select_4 = '''
SELECT author_ratings.author,
       AVG(author_ratings.avg_rating) AS total_rating
FROM
    (SELECT a.author AS author,
            AVG(ra.rating) AS avg_rating
     FROM books AS b
     LEFT JOIN authors AS a ON b.author_id = a.author_id
     LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
     GROUP BY b.book_id,
              a.author_id
     HAVING COUNT(ra.rating_id) > 50
     ) AS author_ratings
GROUP BY author_ratings.author
ORDER BY total_rating DESC
LIMIT 1
; '''
pd.io.sql.read_sql(select_4, con = engine)

,author,total_rating
0,J.K. Rowling/Mary GrandPré,4.283844


**Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [12]:
select_5 = '''
SELECT AVG(users.total_reviews)

FROM
(SELECT username,
        COUNT(review_id) AS total_reviews 
 FROM reviews
 WHERE username IN (
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 50
                    )
 GROUP BY username
 ) AS users

; '''
pd.io.sql.read_sql(select_5, con = engine)

,avg
0,24.333333


### 4. Выводы 

- После 1 января 2000 года вышло 819 книг. 
- Добавили количество обзоров и среднюю оценку для каждой книги
- Издательство Penguin Books является лидером по количеству выпущенных книг с более чем 50 страницами - выпустили 42 книги
- Джоан Роулинг является с самой высокой средней оценкой книг - 4,2838
- Пользователи, которые поставили больше 50 оценок, в среднем оставляют 24 рецензии. 